# Transformer Modeli ile CPU Tahminleri


In [1]:
import time
import math
import torch
import optuna
import numpy as np
import pandas as pd
import torch.nn as nn
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

/home/melih/Documents/Bites_CPU_Analizi/bites_cpu_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Adım 1: Kurulum ve Cihaz Belirleme

In [2]:
# Tüm hesaplamaların yapılacağı cihazı belirle (GPU varsa GPU, yoksa CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hesaplamalar için kullanılacak cihaz: {device}')

Hesaplamalar için kullanılacak cihaz: cuda


## Adım 2: Veri Yükleme ve Ön İşleme

In [3]:
# İşlenmiş veriyi oku
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')

# Tahmin için 3 sütunu da kullan
ts_data = df[['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']].values

# Veriyi 0-1 arasına ölçeklendir
scaler = MinMaxScaler(feature_range=(0, 1))
ts_data_scaled = scaler.fit_transform(ts_data)

# Veriyi eğitim ve test setlerine ayır
train_size = int(len(ts_data_scaled) * 0.8)
train_data, test_data = ts_data_scaled[:train_size], ts_data_scaled[train_size:]

print("Veri yüklendi, ölçeklendi ve bölündü.")

Veri yüklendi, ölçeklendi ve bölündü.


## Adım 3: Zaman Serisi Dizileri Oluşturma

In [ ]:
def create_sequences(data, lookback_window):
    """
    Veriden LSTM'in anlayacağı sıralı diziler oluşturur.
    """
    X, y = [], []
    for i in range(len(data) - lookback_window):
        feature = data[i:(i + lookback_window)]
        target = data[i + lookback_window]
        X.append(feature)
        y.append(target)
        
    # PyTorch'a vermeden önce listeleri tek bir NumPy dizisine çeviriyoruz.
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(y))


lookback = 12  # 1 saatlik veri

X_train, y_train = create_sequences(train_data, lookback)
X_test, y_test = create_sequences(test_data, lookback)

print("Eğitim verisi şekli (X, y):", X_train.shape, y_train.shape)
print("Test verisi şekli (X, y):", X_test.shape, y_test.shape)

Eğitim verisi şekli (X, y): torch.Size([6899, 12, 3]) torch.Size([6899, 3])
Test verisi şekli (X, y): torch.Size([1716, 12, 3]) torch.Size([1716, 3])


## Adım 4: Transformer Modelini Tanımlama ve GPU'ya Taşıma

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        """
        Modelin, sekanstaki pozisyon bilgisini anlaması için sinüs ve kosinüs dalgalarına dayalı bir kodlama matrisi oluşturur.
        """
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Girdi tensörüne, önceden hesaplanmış olan pozisyonel kodlama değerlerini ekler.
        """
        return x + self.pe[:x.size(0), :]

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim=3, d_model=64, nhead=4, num_encoder_layers=2, dim_feedforward=128, dropout=0.1):
        """
        Modelin katmanlarını (girdi kodlayıcı, pozisyonel kodlama, Transformer ve çıktı kod çözücü) tanımlar ve başlatır.
        """
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        """
        Girdi dizisini işler, pozisyon bilgisini ekler, Transformer'dan geçirir ve son adımdan tahmini üretir.
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output[:, -1, :])
        return output

In [7]:
model = TransformerModel().to(device) # Modeli oluşturup doğrudan GPU'ya taşıyoruz

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Transformer Model Mimarisi:")
print(model)

Transformer Model Mimarisi:
TransformerModel(
  (encoder): Linear(in_features=3, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=3, bias=True)
)


## Adım 5: Modeli GPU Üzerinde Eğitme

In [8]:
epochs = 20

print(f"Model {epochs} epoch boyunca '{device}' üzerinde eğitiliyor...")
start_time = time.time()
model.train()

for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        
        # Veriyi her adımda GPU'ya gönder
        seq_gpu = seq.unsqueeze(0).to(device)
        labels_gpu = labels.to(device)
        
        y_pred = model(seq_gpu)
        
        single_loss = loss_function(y_pred.squeeze(0), labels_gpu)
        single_loss.backward()
        optimizer.step()


    print(f'Epoch: {i+1:3} loss: {single_loss.item():10.8f}')

end_time = time.time()
print(f"Eğitim tamamlandı. Süre: {end_time - start_time:.2f} saniye")

Model 20 epoch boyunca 'cuda' üzerinde eğitiliyor...
Epoch:   1 loss: 0.00043528
Epoch:   2 loss: 0.00106709
Epoch:   3 loss: 0.00066721
Epoch:   4 loss: 0.00058624
Epoch:   5 loss: 0.00130270
Epoch:   6 loss: 0.00103841
Epoch:   7 loss: 0.00069153
Epoch:   8 loss: 0.00051256
Epoch:   9 loss: 0.00044512
Epoch:  10 loss: 0.00073691
Epoch:  11 loss: 0.00076246
Epoch:  12 loss: 0.00074132
Epoch:  13 loss: 0.00066207
Epoch:  14 loss: 0.00062959
Epoch:  15 loss: 0.00066161
Epoch:  16 loss: 0.00079631
Epoch:  17 loss: 0.00083605
Epoch:  18 loss: 0.00089486
Epoch:  19 loss: 0.00081481
Epoch:  20 loss: 0.00081347
Eğitim tamamlandı. Süre: 301.48 saniye


## Adım 6: GPU ile Tahmin Yapma ve Sonuçları Değerlendirme

In [ ]:
model.eval()
print("Test verisi üzerinde tahminler yapılıyor...")

test_predictions = []
with torch.no_grad():
    for seq, labels in zip(X_test, y_test):
        # Tahmin için veriyi GPU'ya gönder
        seq_gpu = seq.unsqueeze(0).to(device)
        y_pred = model(seq_gpu)
        # Sonucu CPU'ya geri alıp numpy'a çevir
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

print("Tahminler tamamlandı.")

# Tahminleri ve gerçek değerleri orijinal ölçeğine geri çevir
predictions_array = np.array(test_predictions)
predictions_original_scale = scaler.inverse_transform(predictions_array)
y_test_original_scale = scaler.inverse_transform(y_test.numpy())

def mean_absolute_percentage_error(y_true, y_pred): 
    """
    Gerçek ve tahmin edilen değerler arasındaki ortalama mutlak yüzde hatayı (MAPE), sıfıra bölme hatasını önleyerek hesaplar.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']

print("--- Transformer Model Performans Metrikleri ---")
# Her bir metrik için ayrı ayrı hata hesaplayalım
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    
    # Tüm metrikleri hesapla
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  Ortalama Mutlak Hata (MAE): {mae:.2f}")
    print(f"  Kök Ortalama Kare Hata (RMSE): {rmse:.2f}")
    print(f"  R-Kare (R²): {r2:.2f}")
    print(f"  Ortalama Mutlak Yüzde Hata (MAPE): {mape:.2f}%")

Test verisi üzerinde tahminler yapılıyor...
Tahminler tamamlandı.
--- Transformer Model Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.19
  Kök Ortalama Kare Hata (RMSE): 0.23
  R-Kare (R²): -0.11
  Ortalama Mutlak Yüzde Hata (MAPE): 3.06%

--- Metrik: 'avg_max_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.47
  Kök Ortalama Kare Hata (RMSE): 0.59
  R-Kare (R²): -0.05
  Ortalama Mutlak Yüzde Hata (MAPE): 2.39%

--- Metrik: 'avg_avg_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.31
  Kök Ortalama Kare Hata (RMSE): 0.36
  R-Kare (R²): -0.09
  Ortalama Mutlak Yüzde Hata (MAPE): 3.09%


## Adım 7: Sonuçları Görselleştirme

In [10]:
fig = go.Figure()

for i, column in enumerate(target_columns):
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=y_test_original_scale[:, i],
        mode='lines',
        name=f'Gerçek {column}'
    ))
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=predictions_original_scale[:, i],
        mode='lines',
        name=f'Tahmin {column}',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. Transformer Tahminleri (GPU)',
    xaxis_title='Zaman',
    yaxis_title='Ortalama CPU Yükü (%)'
)

fig.show()

## Adım 8: Hiperparametre Optimizasyonu (Optuna)
Optimizasyon Fonksiyonunu Tanımlama

In [ ]:
# Optimizasyon için eğitim verisini tekrar eğitim ve validasyon olarak ikiye ayırıyoruz
X_train_opt, X_val, y_train_opt, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def objective(trial):
    """
    Optuna'nın her bir deneme için çalıştıracağı fonksiyon.
    """
    # --- 1. Hiperparametre Arama Uzayını Tanımla ---
    d_model = trial.suggest_categorical('d_model', [32, 64, 128])
    nhead = trial.suggest_categorical('nhead', [2, 4, 8])
    num_encoder_layers = trial.suggest_int('num_encoder_layers', 1, 3)
    dim_feedforward = trial.suggest_int('dim_feedforward', 128, 512, step=128)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    
    # --- 2. Modeli Bu Parametrelerle Oluştur ---
    model = TransformerModel(
        input_dim=3,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_encoder_layers,
        dim_feedforward=dim_feedforward
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.MSELoss()

    # --- 3. Modeli Eğit (Daha az epoch ile hızlı bir eğitim) ---
    model.train()
    for i in range(5): # Optimizasyon için daha az epoch yeterli
        for seq, labels in zip(X_train_opt, y_train_opt):
            optimizer.zero_grad()
            seq_gpu = seq.unsqueeze(0).to(device)
            labels_gpu = labels.to(device)
            y_pred = model(seq_gpu)
            loss = loss_function(y_pred.squeeze(0), labels_gpu)
            loss.backward()
            optimizer.step()
            
    # --- 4. Modeli Validasyon Verisiyle Değerlendir ve Skoru Döndür ---
    model.eval()
    val_losses = []
    with torch.no_grad():
        for seq, labels in zip(X_val, y_val):
            seq_gpu = seq.unsqueeze(0).to(device)
            labels_gpu = labels.to(device)
            y_pred = model(seq_gpu)
            loss = loss_function(y_pred.squeeze(0), labels_gpu)
            val_losses.append(loss.item())
            
    # Ortalam validasyon loss'unu döndür (Optuna bunu minimize etmeye çalışacak)
    return np.mean(val_losses)

## Adım 9: Optimizasyon Çalışmasını Başlatma

In [12]:
# Bir optimizasyon "çalışması" oluştur. direction='minimize' en düşük loss'u aradığımızı belirtir.
study = optuna.create_study(direction='minimize')

# Optimizasyonu başlat. n_trials, kaç farklı kombinasyon deneneceğini belirtir.
# Örnek olarak 20 deneme yapalım. Bu sayı artırılabilir.
study.optimize(objective, n_trials=20)

print("\nOptimizasyon tamamlandı!")
print("En iyi deneme:")
print("  Değer (min validation loss): ", study.best_value)
print("  En iyi parametreler: ", study.best_params)

[I 2025-07-29 14:46:53,108] A new study created in memory with name: no-name-24a3e07d-c11f-41f5-beab-efea464d897f
[I 2025-07-29 14:47:53,141] Trial 0 finished with value: 0.0021441914412734797 and parameters: {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 128, 'lr': 0.00038471158464620367}. Best is trial 0 with value: 0.0021441914412734797.
[I 2025-07-29 14:48:30,585] Trial 1 finished with value: 0.002199095875076584 and parameters: {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 1, 'dim_feedforward': 128, 'lr': 0.0007177543887266996}. Best is trial 0 with value: 0.0021441914412734797.
[I 2025-07-29 14:49:57,510] Trial 2 finished with value: 0.0020893342086936544 and parameters: {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 3, 'dim_feedforward': 128, 'lr': 0.00014648700094958353}. Best is trial 2 with value: 0.0020893342086936544.
[I 2025-07-29 14:51:21,255] Trial 3 finished with value: 0.0021142745292941664 and parameters: {'d_model': 32, 'nhead': 8, 'nu


Optimizasyon tamamlandı!
En iyi deneme:
  Değer (min validation loss):  0.001879004345717285
  En iyi parametreler:  {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 384, 'lr': 0.00019162951777090744}


## Adım 10: En İyi Parametrelerle Nihai Modelin Eğitilmesi ve Değerlendirilmesi

In [17]:
# Optuna'nın bulduğu en iyi parametreleri bir sözlüğe kopyalayalım
best_params = {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 384, 'lr': 0.00019162951777090744}

# Bu en iyi parametrelerle nihai modelimizi oluşturalım ve GPU'ya taşıyalım
final_model = TransformerModel(
    input_dim=3,
    d_model=best_params['d_model'],
    nhead=best_params['nhead'],
    num_encoder_layers=best_params['num_encoder_layers'],
    dim_feedforward=best_params['dim_feedforward']
).to(device)

# Optimizer'ı da en iyi öğrenme oranı (lr) ile ayarlayalım
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['lr'])
loss_function = nn.MSELoss()

print("--- En İyi Parametrelerle Nihai Model Oluşturuldu ---")
print(final_model)

--- En İyi Parametrelerle Nihai Model Oluşturuldu ---
TransformerModel(
  (encoder): Linear(in_features=3, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=384, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=384, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=128, out_features=3, bias=True)
)


In [18]:
epochs = 20

print(f"Nihai model {epochs} epoch boyunca '{device}' üzerinde eğitiliyor...")
start_time = time.time()
final_model.train()

for i in range(epochs):
    for seq, labels in zip(X_train, y_train): # Tam X_train ve y_train kullanılıyor
        optimizer.zero_grad()
        
        seq_gpu = seq.unsqueeze(0).to(device)
        labels_gpu = labels.to(device)
        
        y_pred = final_model(seq_gpu)
        
        single_loss = loss_function(y_pred.squeeze(0), labels_gpu)
        single_loss.backward()
        optimizer.step()

    print(f'Epoch: {i+1:3} loss: {single_loss.item():10.8f}')

end_time = time.time()
print(f"Nihai eğitim tamamlandı. Süre: {end_time - start_time:.2f} saniye")

Nihai model 20 epoch boyunca 'cuda' üzerinde eğitiliyor...
Epoch:   1 loss: 0.00056004
Epoch:   2 loss: 0.00041221
Epoch:   3 loss: 0.00066752
Epoch:   4 loss: 0.00037618
Epoch:   5 loss: 0.00093079
Epoch:   6 loss: 0.00068398
Epoch:   7 loss: 0.00167319
Epoch:   8 loss: 0.00078316
Epoch:   9 loss: 0.00094579
Epoch:  10 loss: 0.00139654
Epoch:  11 loss: 0.00108885
Epoch:  12 loss: 0.00166983
Epoch:  13 loss: 0.00111512
Epoch:  14 loss: 0.00109776
Epoch:  15 loss: 0.00091213
Epoch:  16 loss: 0.00138555
Epoch:  17 loss: 0.00087372
Epoch:  18 loss: 0.00139493
Epoch:  19 loss: 0.00086531
Epoch:  20 loss: 0.00110449
Nihai eğitim tamamlandı. Süre: 418.64 saniye


## Adım 11: GPU ile Tahmin Yapma ve Sonuçları Değerlendirme

In [19]:
final_model.eval()
print("Nihai model ile test verisi üzerinde tahminler yapılıyor...")

# Tahmin yap
test_predictions = []
with torch.no_grad():
    for seq, labels in zip(X_test, y_test):
        seq_gpu = seq.unsqueeze(0).to(device)
        y_pred = final_model(seq_gpu)
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

# Ölçeği geri çevir
predictions_array = np.array(test_predictions)
predictions_original_scale = scaler.inverse_transform(predictions_array)
y_test_original_scale = scaler.inverse_transform(y_test.numpy())

# Metrikleri hesapla
print("\n--- Optimize Edilmiş Transformer Modelinin Nihai Performans Metrikleri ---")
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  Ortalama Mutlak Hata (MAE): {mae:.2f}")
    print(f"  Kök Ortalama Kare Hata (RMSE): {rmse:.2f}")
    print(f"  R-Kare (R²): {r2:.2f}")
    print(f"  Ortalama Mutlak Yüzde Hata (MAPE): {mape:.2f}%")

Nihai model ile test verisi üzerinde tahminler yapılıyor...

--- Optimize Edilmiş Transformer Modelinin Nihai Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.18
  Kök Ortalama Kare Hata (RMSE): 0.22
  R-Kare (R²): 0.01
  Ortalama Mutlak Yüzde Hata (MAPE): 2.89%

--- Metrik: 'avg_max_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.30
  Kök Ortalama Kare Hata (RMSE): 0.37
  R-Kare (R²): 0.58
  Ortalama Mutlak Yüzde Hata (MAPE): 1.51%

--- Metrik: 'avg_avg_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.22
  Kök Ortalama Kare Hata (RMSE): 0.27
  R-Kare (R²): 0.37
  Ortalama Mutlak Yüzde Hata (MAPE): 2.26%


## Adım 12: Sonuçları Görselleştirme

In [20]:
fig = go.Figure()
for i, column in enumerate(target_columns):
    fig.add_trace(
        go.Scatter(
            x=df.index[train_size+lookback:],
            y=y_test_original_scale[:, i], 
            mode='lines', 
            name=f'Gerçek {column}'
            ))
    
    fig.add_trace(
        go.Scatter(
            x=df.index[train_size+lookback:], 
            y=predictions_original_scale[:, i], 
            mode='lines', 
            name=f'Optimize Tahmin {column}', 
            line=dict(dash='dash')
            ))
    
fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. Optimize Edilmiş Transformer Tahminleri', 
    xaxis_title='Zaman', 
    yaxis_title='Ortalama CPU Yükü (%)'
    )

fig.show()